In [3]:
import requests
import pandas as pd
import logging
from textblob import TextBlob
from datetime import datetime, timedelta
import time

# Initialize logger
logging.basicConfig(filename="trading_logs.log", level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s")

# Configuration
NEWS_API_KEY = "bc9bd08322254932a5ee5c77b192f35c"  # Your provided API key
BASE_URL = "https://newsapi.org/v2/everything"
WATCHLIST = [
    "AAPL",  # Apple
    "GOOGL",  # Alphabet (Google)
    "MSFT",  # Microsoft
    "TSLA",  # Tesla
    "AMZN",  # Amazon
    "META",  # Meta (Facebook)
    "NFLX",  # Netflix
    "NVDA",  # Nvidia
    "BA",  # Boeing
    "SPY",  # S&P 500 ETF (tracks major index)
    "AMD",  # Advanced Micro Devices
    "DIS",  # Walt Disney
    "V",  # Visa
    "MA",  # Mastercard
    "PYPL",  # PayPal
    "INTC",  # Intel
    "SQ",  # Square
    "TWTR",  # Twitter (now X)
    "BABA",  # Alibaba
    "UBER",  # Uber Technologies
    "RIOT",  # Riot Blockchain
    "LI",  # Li Auto
    "BYND",  # Beyond Meat
    "CRM",  # Salesforce
    "COST",  # Costco Wholesale
    "WMT",  # Walmart
    "KO",  # Coca-Cola
    "PEP",  # PepsiCo
    "PFE",  # Pfizer
    "MRNA",  # Moderna
    "XOM",  # Exxon Mobil
    "CVX",  # Chevron
    "JNJ",  # Johnson & Johnson
    "MCD",  # McDonald's
    "GS",  # Goldman Sachs
    "JPM",  # JPMorgan Chase
    "AMAT",  # Applied Materials
    "T",  # AT&T
    "VZ",  # Verizon Communications
]
  # List of stocks to monitor
SENTIMENT_THRESHOLD = 0.2  # Adjust threshold for sentiment analysis

# Portfolio simulation
portfolio = {stock: 0 for stock in WATCHLIST}  # Track stock holdings
cash = 1000000  # Starting cash balance in USD
TRADE_LOG = []

def fetch_news(stock_symbol):
    """Fetch news articles for a specific stock."""
    url = f"{BASE_URL}?q={stock_symbol}&apiKey={NEWS_API_KEY}&language=en"
    response = requests.get(url)
    data = response.json()
    articles = data.get("articles", [])
    return articles

def analyze_sentiment(text):
    """Perform sentiment analysis on the news headline/content."""
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

def simulate_trade(stock, sentiment, price):
    """Simulate buy or sell trades based on sentiment."""
    global cash
    position = portfolio.get(stock, 0)
    qty = 10  # Fixed quantity for each trade

    if sentiment > SENTIMENT_THRESHOLD and cash >= qty * price:
        # Buy stock
        portfolio[stock] += qty
        cash -= qty * price
        log_trade(stock, "buy", qty, price)
        logging.info(f"Bought {qty} of {stock} at {price}. New cash: {cash}")
    elif sentiment < -SENTIMENT_THRESHOLD and position >= qty:
        # Sell stock
        portfolio[stock] -= qty
        cash += qty * price
        log_trade(stock, "sell", qty, price)
        logging.info(f"Sold {qty} of {stock} at {price}. New cash: {cash}")

def log_trade(stock, action, qty, price):
    """Log a trade to the trade log."""
    TRADE_LOG.append({
        "timestamp": datetime.now(),
        "stock": stock,
        "action": action,
        "quantity": qty,
        "price": price
    })

def get_mock_price(stock):
    """Simulate stock price retrieval (replace with real API)."""
    return round(100 + (2 * len(stock)) + (datetime.now().second % 10), 2)  # Mock price

def monitor_stocks():
    """Monitor stocks continuously for news and make trading decisions."""
    while True:
        for stock in WATCHLIST:
            articles = fetch_news(stock)
            for article in articles:
                title = article["title"]
                description = article.get("description", "")
                content = title + " " + (description if description else "")  # Handle None
                sentiment = analyze_sentiment(content)
                price = get_mock_price(stock)
                logging.info(f"Analyzed sentiment for {stock}: {sentiment}. Content: {title}")
                simulate_trade(stock, sentiment, price)

        # Print a summary of the current state before sleeping
        print(f"Completed one cycle of monitoring. Portfolio: {portfolio}, Cash: {cash}")
        logging.info("Completed one cycle of monitoring. Sleeping for 10 minutes.")

        time.sleep(100)  # Sleep for 10 minutes before next cycle


if __name__ == "__main__":
    try:
        logging.info("Starting stock monitoring system...")
        monitor_stocks()
    except Exception as e:
        logging.error(f"Error occurred: {e}")


Completed one cycle of monitoring. Portfolio: {'AAPL': 530, 'GOOGL': 460, 'MSFT': 420, 'TSLA': 310, 'AMZN': 510, 'META': 110, 'NFLX': 250, 'NVDA': 320, 'BA': 250, 'SPY': 170, 'AMD': 360, 'DIS': 290, 'V': 270, 'MA': 240, 'PYPL': 90, 'INTC': 180, 'SQ': 300, 'TWTR': 0, 'BABA': 120, 'UBER': 100, 'RIOT': 260, 'LI': 320, 'BYND': 10, 'CRM': 370, 'COST': 220, 'WMT': 410, 'KO': 350, 'PEP': 240, 'PFE': 150, 'MRNA': 70, 'XOM': 210, 'CVX': 50, 'JNJ': 90, 'MCD': 100, 'GS': 350, 'JPM': 270, 'AMAT': 70, 'T': 140, 'VZ': 0}, Cash: 1020


KeyboardInterrupt: 